In [2]:
!pip install -q pypdf transformers einops accelerate langchain bitsandbytes sentence-transformers llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [3]:
!pip install -q llama-index-llms-huggingface
!pip install -q langchain-huggingface
!pip install -q llama-index-embeddings-langchain

In [4]:
import importlib.metadata
print(importlib.metadata.version("llama-index"))

0.12.30


In [11]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts import PromptTemplate

In [6]:
documents = SimpleDirectoryReader(input_dir="./data").load_data()
documents

[Document(id_='740bb020-b15f-46b4-ac9f-5a84b6e0c6dd', embedding=None, metadata={'page_label': '1', 'file_name': 'Attention.pdf', 'file_path': '/content/data/Attention.pdf', 'file_type': 'application/pdf', 'file_size': 569417, 'creation_date': '2025-04-16', 'last_modified_date': '2025-04-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of

In [21]:
documents[0]

Document(id_='740bb020-b15f-46b4-ac9f-5a84b6e0c6dd', embedding=None, metadata={'page_label': '1', 'file_name': 'Attention.pdf', 'file_path': '/content/data/Attention.pdf', 'file_type': 'application/pdf', 'file_size': 569417, 'creation_date': '2025-04-16', 'last_modified_date': '2025-04-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of 

In [ ]:
# RichPromptTemplate - latest-style for building jinja-style prompts with variables and logic
# PromptTemplate - older-style simple templating for building prompts with a single f-string

In [ ]:
system_prompt = """
<|system|>
You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided.
<|end|>
""".strip()
# Llama2 prompt format
query_wrapper_prompt = RichPromptTemplate("<|user|>{query_str}<|end|>\n<|assistant|>")
query_wrapper_prompt

RichPromptTemplate(metadata={}, template_vars=[], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template_str='<|user|>{query_str}<|end|>\n<|assistant|>')

In [12]:
system_prompt = """You are a Q&A assistant. Your goal is to answer question as accurately as possible based on instructions and context provided."""
# Llama3.2 prompt format
query_wrapper_prompt = PromptTemplate(
    f"""<|start_header_id|>system<|end_header_id|>
{system_prompt}<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{{{{query_str}}}}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""
)
query_wrapper_prompt

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='<|start_header_id|>system<|end_header_id|>\nYou are a Q&A assistant. Your goal is to answer question as accurately as possible based on instructions and context provided.<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n{{query_str}}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

In [9]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `LlamaIndex_llama` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `LlamaInd

In [40]:
import torch
from transformers import BitsAndBytesConfig

# Define quantization config for 8-bit
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=512, #256
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-3.2-1B-Instruct",
    model_name="meta-llama/Llama-3.2-1B-Instruct",
    device_map="auto",
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": bnb_config
    }
)

In [14]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
#from llama_index.core import ServiceContext
from llama_index.core import Settings
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [41]:
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024

In [42]:
index = VectorStoreIndex.from_documents(documents)

In [43]:
index

In [44]:
query_engine = index.as_query_engine(similarity_top_k=3)

In [ ]:
response = query_engine.query("what is attention is all you need?")

In [33]:
response.response

'The query is: "what is attention is all you need?"\n\nThe answer is: "Attention Is All You Need" is a paper written by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin, published in 2017.'

In [37]:
query_engine = index.as_query_engine(similarity_top_k=20)

In [38]:
response = query_engine.query("what is attention is all you need?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [39]:
response.response

'Based on the provided context, I will refine the original answer to better answer the query.\n\nOriginal Answer: "The refined answer is: \'Attention Is All You Need\' is a paper presented by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin at the 31st Conference on Neural Information Processing Systems (NIPS) in 2017."\n\nRefined Answer: "Attention Is All You Need" is a paper presented by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin at the 31st Conference on Neural Information Processing Systems (NIPS) in 2017. This paper introduced the concept of attention mechanisms in deep learning, which revolutionized the field of object detection by enabling models to focus on specific parts of the image that are most relevant to the task at hand. The refined answer highlights the significance of attention mechanisms in object detection and t

In [47]:
query_engine = index.as_query_engine(similarity_top_k=10)

In [48]:
response = query_engine.query("what is attention is all you need?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [49]:
response.response

'The refined answer to the query "what is attention is all you need?" is:\n\nAttention is all you need because it allows for the efficient processing of long-range dependencies in neural networks, making it faster and more computationally efficient than recurrent layers, which require O(n) sequential operations.'